In [ ]:
# export
# default_exp modular_unet
import torch
from torch import nn
from torch.nn import functional as F
from collections import OrderedDict

from fastcore.dispatch import patch
from fastcore.basics import store_attr

In [ ]:
# export
import sys
sys.path.append('..')
from attention_unet.blocks import UnetBlock, ConvLayer, SqueezeExpand, BasicResBlock
from attention_unet.utils import hasattrs

In [ ]:
# export
class UNet(nn.Module):
    " Simple 3D UNet "
    channels = 32, 64, 128, 256, 512
    stride = 2, 2, 2, 2, 2
    blocks = 1, 2, 2, 2, 2
    n_layers = 5

    def __init__(self, in_c, n_classes, act=nn.ReLU, norm=nn.BatchNorm3d, debug=True):
        super(UNet, self).__init__()
        self.debug=debug
        original_in_c = in_c
        for i in range(self.n_layers):
            setattr(self, f'down_layer_{i}', res_blocks(in_c, self.channels[i],
                                                        self.stride[i], self.blocks[i],
                                                        act=act, norm=norm))
            in_c = self.channels[i]

        self.middle_conv = SqueezeExpand(in_c, 0.2)

        for i in reversed(range(self.n_layers)):
            s_c = self.channels[i-1] if i > 0 else original_in_c
            setattr(self, f'up_layer_{i}', UnetBlock(self.channels[i], s_c, act=act, norm=norm))

        self.final_conv = ConvLayer((s_c + self.channels[i]//2) // 2, n_classes, ks = 1)

        if self.debug:
            self.report = {'in_max': [],
                           'in_mean': [],
                           'in_min': [],
                           'out_max': [],
                           'out_mean': [],
                           'out_min': []}


    def forward(self, x):

        if (x!=x).any():
            print('NaN detected in input. Saved input to self.input.')
            self.input = x

        if self.debug:
            self.report['in_max'].append(float(x.max().detach().cpu().numpy()))
            self.report['in_mean'].append(float(x.mean().detach().cpu().numpy()))
            self.report['in_min'].append(float(x.min().detach().cpu().numpy()))

        sz = x.shape[-3:]
        # downsampling
        s = [x] # collect outouts for skip connections
        for i in range(self.n_layers):
            x = getattr(self, f'down_layer_{i}')(x)
            s.append(x)

        # middle conv missing
        x = self.middle_conv(x)

        # upsampling
        for i in reversed(range(self.n_layers)):
            x = getattr(self, f'up_layer_{i}')(x, s[i])
            # len(s) == (self.n_layers + 1), so s[i] will yield the right skip connection

        # final layer
        x = self.final_conv(x)

        # final resize
        if x.shape[-3:] != sz:
            x = F.interpolate(x, sz, mode='nearest')

        if self.debug:
            self.report['out_max'].append(float(x.max().detach().cpu().numpy()))
            self.report['out_mean'].append(float(x.mean().detach().cpu().numpy()))
            self.report['out_min'].append(float(x.min().detach().cpu().numpy()))
        return x

In [ ]:
# export
class ModularUNet(nn.Module): 
    " Modular 3D UNet "

    def __init__(self, 
                 in_c:int, # number of input channels
                 n_classes:int, # number of output channels / number of classes
                 norm=nn.BatchNorm3d, # type of batch nornalization
                 act=nn.ReLU, # activation function
                 **kwargs # further arguments for ConvLayer
                ): 
        super(ModularUNet, self).__init__()
        store_attr(store_args=True) # saves all attributes to self
        
        hasattrs(self, ('channels', 'kernel_size', 'stride', 'padding', 'n_layers', 'n_blocks'), do_raise=True)
        
        # encoder layers (downsampling)
        original_in_c = in_c
        for i in range(self.n_blocks):
            setattr(self, 
                    f'encoder_block_{i}', 
                    self.create_encoder_block(in_c, self.channels[i], self.kernel_size[i], 
                                              self.stride[i], self.padding[i], self.n_layers[i], **kwargs)
                   )
            in_c = self.channels[i]
        
        # skip blocks (applied to gated inputs from skip connections before passed to extra layer or upsampling)
        for i in range(self.n_blocks-1): # the last skip_block is essentially the middle conv
            setattr(self, 
                    f'skip_block_{i}', 
                    self.create_skip_block(self.channels[i], **kwargs)
                   )
            
        # Middle Conv
        self.middle_block = self.create_middle_block(self.channels[-1], **kwargs) 
            
        # Decoder (upsampling)
        for i in reversed(range(self.n_blocks)): 
            s_c = self.channels[i-1] if i > 0 else original_in_c
            setattr(self, f'decoder_block_{i}', self.create_decoder_block(in_c=self.channels[i], s_c=s_c, **kwargs))
            
            in_c = (s_c + self.channels[i]//2) // 2
            setattr(self, f'extra_after_decoder_block_{i}', 
                    self.create_extra_after_decoder_block(in_c=in_c, out_c=n_classes, **kwargs))
            
        # Final layer (n_channels -> n_classes)
        if not isinstance(self.extra_after_decoder_block_0, nn.Identity): 
            in_c = n_classes * self.n_blocks
        self.final_block = self.create_final_block(in_c, n_classes)
     
    # Each block is created with a `create`_block` function, which passes arguments to the `layer` function
    # The `layer` function can either be patched to ModularUnet or added in a subclass.
    def create_encoder_block(self, in_c, out_c, ks, stride, padding, n_layers, **kwargs): 
        " Create `n_layers` instances of `encoder_layer` "        
        assert hasattr(self, 'encoder_layer'), self._not_implemented_error('encoder_layer')
        layers = OrderedDict([('layer_0', self.encoder_layer(in_c=in_c, out_c=out_c, ks=ks, stride=stride, 
                                                            padding=padding, **kwargs))])
        if n_layers == 1: return nn.Sequential(layers)
        for i in range(n_layers - 1):
            layers[f'layer_{i+1}'] = self.encoder_layer(in_c=out_c, out_c=out_c, ks=ks, stride=stride, 
                                                        padding=padding, **kwargs)
        return nn.Sequential(layers)

    def create_skip_block(self, in_c, **kwargs):
        " Build skip blocks "
        assert hasattr(self, 'skip_layer'), self._not_implemented_error('skip_layer')
        return self.skip_layer(in_c=in_c, **kwargs)
    
    def create_middle_block(self, in_c, **kwargs): 
        " Build middle layer "
        assert hasattr(self, 'middle_layer'), self._not_implemented_error('middle_layer')
        return  self.middle_layer(in_c=in_c, **kwargs)
    
    def create_decoder_block(self, in_c, s_c, **kwargs): 
        " Build decoder layers "
        assert hasattr(self, 'decoder_layer'), self._not_implemented_error('decoder_layer')
        return self.decoder_layer(in_c=in_c, s_c=s_c, **kwargs)
    
    def create_extra_after_decoder_block(self, in_c, out_c, **kwargs): 
        " Build extra layers applied to each output of the decoder blocks"
        assert hasattr(self, 'extra_after_decoder_layer'), self._not_implemented_error('extra_after_decoder_layer')
        return self.extra_after_decoder_layer(in_c=in_c, out_c=out_c, **kwargs)
    
    def create_final_block(self, in_c, out_c, **kwargs):
        " Build last layer. Defaults to simple ConvLayer if no further arguments are given. "
        if hasattr(self, 'final_layer'): 
            return self.final_layer(in_c=in_c, out_c=out_c)
        return ConvLayer(in_c, out_c, ks=1, **kwargs, act=None, norm=None)            
    
    # implement forward functions for each part of the Modular UNet
    def forward_encoder(self, x): 
        " x -> [x1, x2, .. , xn]. Store x1, .. , xn, return xn"
        # downsampling
        self.s = [x] # collect outouts for skip connections
        for i in range(self.n_blocks): 
            x = getattr(self, f'encoder_block_{i}')(x)
            self.s.append(x) # store intermediate output as skip connection
        return x
    
    def forward_skip(self):
        "x1 -> x1', x2 -> x2', .. , xn -> xn'"
        for i in range(self.n_blocks - 1):
            # self.s[0] is the input, not the output from the first encoder block
            # so self.s[1] will yield the output from encoder_block_0 for skip_block_0
            self.s[i+1] = getattr(self, f'skip_block_{i}')(self.s[i+1])
    
    def forward_middle(self, x):
        " x -> x "
        return self.middle_block(x)

    def forward_decoder(self, x):
        "x + x1 -> x_up1, x_up1 + x2 -> x_up2, .. , x_up(n-1) + x_n -> x_upn"
        # upsampling
        for i in reversed(range(self.n_blocks)):
            x = getattr(self, f'decoder_block_{i}')(x, self.s[i]) 
            # len(s) == (self.n_layers + 1), so s[i] will yield the right skip connection
            self.s[i] = (x) # store output from U-Net Block for use in e.g. Deep Supervision

        # self.s = [x_up1, x_up2, .., x_upn, xn], where xn is the output from `self.middle_block`
        # xn is not needed anymore and can be removed
        self.s = self.s[:-1]
        return x
    
    def forward_extra_after_decoder(self):
        "x_up1 -> x_up1', x_up2 -> x_up2', .. , x_upn -> x_upn'"
        for i in reversed(range(self.n_blocks)):
            # self.s[0] is the out, not the output from the last decoder block (decoder_block_0)
            # so self.s[n_blocks]  is the output of the first decoder block (decoder_block_(n_blocks))
            self.s[i] = getattr(self, f'extra_after_decoder_block_{i}')(self.s[i])
        
    def forward_final(self, x):
        "x -> classes"
        # final layer
        return self.final_block(x)
        
    def forward(self, x):
        sz = x.shape[-3:] # store size for resizing
        
        x = self.forward_encoder(x)
        self.forward_skip()
        x = self.forward_middle(x)
#         x = self.forward_extra_after_skip(x) # maybe added later. A global layer with access to all tensors sored in self.s
        x = self.forward_decoder(x)
        self.forward_extra_after_decoder()
        # if `forward_extra_after_decoder` changed self.s, then the combined tensors of 
        # self.s are passed to the final block. For this they need to be resized and concatenated
        if not isinstance(self.extra_after_decoder_block_0, nn.Identity): 
            self.s = [F.interpolate(s, sz, mode='nearest') for s in self.s]
            x = torch.cat(self.s, 1)
        x = self.forward_final(x)
        
        # final resize
        if x.shape[-3:] != sz: 
            x = F.interpolate(x, sz, mode='nearest')
        return x
            
    def _not_implemented_error(self, layer_name): 
        msg1, msg2 = 'For example:\n@patch\ndef', '(self:ModularUNet, **kwargs): return' 
        if layer_name == 'encoder_layer': 
            patch = 'BasicResBlock(**kwargs)'
        elif layer_name == 'middle_layer': 
            patch = 'SqueezeExpand(**kwargs, se_ratio=0.2)'
        elif layer_name == 'decoder_layer': 
            patch = 'UnetBlock(**kwargs)'
        else: patch='nn.Identity()'
        raise NotImplementedError(f'self.{layer_name} is not implemented for `ModularUNet`. '
                                  f'It can be added using the `@patch` decorator or by subclassing `ModularUNet`'
                                  f'{msg1} {layer_name} {msg2} {patch}'
                                 )

## Create a UResNet

In [ ]:
# export
class UResNet18(ModularUNet):
    channels = 32, 64, 128, 256, 512
    kernel_size = 3, 3, 3, 3, 3
    stride = 2, 2, 2, 2, 2
    padding = 'auto', 'auto', 'auto', 'auto', 'auto'
    n_layers = 1, 2, 2, 2, 2
    n_blocks = 5
    
    def encoder_layer(self, **kwargs): return BasicResBlock(**kwargs)
    def middle_layer(self, **kwargs): return SqueezeExpand(**kwargs, se_ratio=0.2)
    def skip_layer(self, **kwargs): return nn.Identity()
    def decoder_layer(self, **kwargs): return UnetBlock(**kwargs)
    def extra_after_decoder_layer(self, **kwargs): return nn.Identity()
    def final_layer(self, **kwargs): return BasicResBlock(**kwargs)

In [ ]:
UResNet18(3,3)(torch.randn(2,3,10,25,25)).shape

torch.Size([2, 3, 10, 25, 25])

In [ ]:
class UResNet34(UResNet18):
    n_layers = 1, 3, 4, 6, 3

In [ ]:
UResNet34(3,3)(torch.randn(2,3,10,25,25)).shape

torch.Size([2, 3, 10, 25, 25])

In [ ]:
class UResNet18WithAttention(UResNet18):
    pass

In [ ]:
@patch
def decoder_layer(self:UResNet18WithAttention, **kwargs): return UnetBlock(spatial_attention=True, **kwargs)

In [ ]:
m = UResNet18WithAttention(3,3)
assert hasattr(m.decoder_block_1, 'sa'), 'No spatial attnetion layer in decoder block'

In [ ]:
m(torch.randn(2,3,10,25,25)).shape

torch.Size([2, 3, 10, 25, 25])

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted blocks.ipynb.
Converted index.ipynb.
Converted modular_unet.ipynb.
Converted utils.ipynb.
